# Part 1 - WML Federated Learning with MNIST for Admin 

IBM Federated Learningを利用すれば、実際にデータを共有することなく、複数のソースからのデータを組み合わせて、集合データからモデルをトレーニングすることができます。  
これにより、企業はセキュリティのためにリソースを委譲することなく、他社とデータのトレーニングを行うことができます。  
もう一つの利点は、リモートデータを一箇所に集中管理する必要がないため、潜在的に大規模なデータセットを移動させる必要がないことです。  
このノートブックでは、Pythonクライアントを使用して連携学習を開始する方法をデモします。  
Federated Learningのセットアップ、用語、UIからのFederated Learningの実行などの詳細については、[Federated Learningのドキュメント](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fed-lea.html?audience=wdp)を参照してください。




### 学習目標

Part11 - WML Federated Learning with MNIST for Adminノートブックを終了すると、以下のようなことができるようになります。

- 未学習モデルをロードする
- リモートトレーニングシステムの作成
- トレーニング・ジョブの開始

このノートを完成させたら、Part 2 - WML Federated Learning with MNIST for Partyを開いてください。

<div class="alert-block alert-info">このノートはFederated Learning実験の管理者が実行することを想定しています。</div>

## 目次

- [1.前提条件](#prequisites)
    - [1.1 変数の定義](#var)
    - [1.2 ライブラリのインポート](#libraries)
    - [1.3 クラウド認証トークンの取得](#cloud-auth)
- [2.モデルの読み込み](#load)
    - [2.1 未学習モデル資産の作成](#model)
    - [2.2 初期モデルのアップロード](#init)
- [3.リモートトレーニングシステムの作成](#rts)
    - [3.1 リモートトレーニングシステム資産の作成](#create-rts)
- [4.FLトレーニングジョブを作成](#fl-job)
    - [4.1 トレーニングジョブのステータス取得](#status)

- Cloud Object Storage バケット。IBM Cloud Services](https://cloud.ibm.com/resources)のCloud Object Storageインスタンスから作成できます。
- HMACを有効にしたCloud Object Storageサービスクレデンシャル。作成するには、**Cloud Object Storage < Service Credentials** に移動し、**New credentials** をクリックします。ライターアクセスを与えて、**Advanced options**をクリックし、*Include HMAC Credential*をオンにします。
- IAM API キー。新しいキーを作成するには、[IBM Cloud ホームページ](https://cloud.ibm.com) にアクセスします。アカウントで、**Manage < IAM < API Keys**に移動します。**Create an IBM Cloud API Key**をクリックします。


<a id = "var"></a>
### 1.1 変数の定義

#### 共通設定

In [ ]:
API_VERSION = "2019-10-25"
IAM_TOKEN_URL = "https://iam.cloud.ibm.com/oidc/token"
IAM_APIKEY = "xxxx"

# Manage < IAM < Usersから取得し、URLを確認します。ユーザーIDはIBMid-<xxx>の形式になっているはずです。
CLOUD_USERID = "xxxx"

# Studioプロジェクトに入ってURLを確認することで取得できます。
PROJECT_ID ="xxxx"

#### WML設定

In [ ]:
# WMLのロケーションにより変更します
WML_SERVICES_HOST = "us-south.ml.cloud.ibm.com"
WML_SERVICES_URL = "https://" + WML_SERVICES_HOST

#### COS設定
次のリンクからエンドポイントのURLを探します:     
https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-endpoints

In [ ]:
# COS設定
COS_ENDPOINT_URL = "https://s3.us-south.cloud-object-storage.appdomain.cloud" 

# HMACサービスの資格情報からアクセスキーIDとSECRET_ACCESS_KEYをCOS用に取得します
COS_ACCESS_KEY_ID = "xxxx"
COS_SECRET_ACCESS_KEY = "xxxx"

# COS bucket名
COS_BUCKET = "xxxx" 


<a id = "libraries"></a>
### 1.2 ライブラリのインポート

In [ ]:
import urllib3
import requests
import json
from string import Template

urllib3.disable_warnings()

<a id = "cloud-auth"></a>
### 1.3 クラウド認証トークンの取得

In [ ]:
payload = "grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey=" + IAM_APIKEY
token_resp = requests.post(IAM_TOKEN_URL ,
                          headers={"Content-Type": "application/x-www-form-urlencoded"}, 
                          data = payload,
                          verify=True)

In [ ]:
# 結果確認
print(token_resp)

token = "Bearer " + json.loads(token_resp.content.decode("utf-8"))["access_token"]
print("WS token: %s " % token)

<a id = "load"></a>
## 2. モデルの読み込み

Federated Learningで作業するためには、未訓練のモデルアセットが必要です。  
このチュートリアルでは、未学習の Tensorflow 2 Keras モデルを提供します。  
Federated Learning は Scikit-learn と Tensorflow 2 をサポートしています。  
さらに、IBM のドキュメントには、Federated Learning のための非訓練モデルの設定方法についての詳細が記載されています。参照してください。

- [Scikit-learnモデルの設定](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fl-imp-det.html?audience=wdp&context=cpdaas#sklearn-config)
- [Tensorflow 2のモデル構成](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fl-imp-det.html?audience=wdp&context=cpdaas#tf-config)


<a id = "model"></a>
### 2.1 未学習モデル資産の作成

プロジェクト内に未訓練のモデルアセットを作成します。

In [ ]:
base_model_save_payload = Template("""
{
    "name": "Untrained MNIST Model",
    "type": "tensorflow_2.1 ",
    "software_spec": {
        "name": "default_py3.7"
    },
    "custom": {
        "untrained" : true
    },
    "project_id": "$projectId"
}
""").substitute(projectId = PROJECT_ID)

print ("Model base save payload: %s" % base_model_save_payload)

base_model_save_resp = requests.post(WML_SERVICES_URL + "/ml/v4/models",
                                     headers={"Content-Type": "application/json",
                                              "Authorization": token},
                                     params={"version": API_VERSION},
                                     data=base_model_save_payload,
                                     verify=False)

print("Base model save response " + base_model_save_resp.content.decode("utf-8"))

In [ ]:
base_model_id = json.loads(base_model_save_resp.content.decode("utf-8"))["metadata"]["id"]
print("Saved model id: %s" % base_model_id)

<a id = "init"></a>
### 2.2 初期モデルのアップロード

初期モデルをプロジェクトにアップロードする必要があります。  
このチュートリアルでは、訓練されていないモデルの例が提供されており、[GitHub](https://github.com/IBMDataScience/sample-notebooks/blob/master/Files/tf_mnist_model.zip).からダウンロードすることができます。

In [ ]:
base_model_content_resp = requests.get("https://github.com/IBMDataScience/sample-notebooks/raw/master/Files/tf_mnist_model.zip",
                                       headers={"Content-Type": "application/octet-stream"})

base_model_content_uri = "/ml/v4/models/"+ base_model_id + "/content"
print("Host URL = " + WML_SERVICES_URL + base_model_content_uri)

base_model_put_resp = requests.put(WML_SERVICES_URL + base_model_content_uri,
                                   headers={"Content-Type": "application/json",
                                            "Authorization": token},
                                   params={"version": API_VERSION,
                                           "project_id": PROJECT_ID,
                                           "content_format": "native"},
                                   data=base_model_content_resp.content,
                                   verify=False)

print("Response status for upload base model  " + base_model_id + " : "+ str(base_model_put_resp.status_code))
print("Create library content: %s"  % base_model_put_resp.content.decode("utf-8"))

<a id = "create-rts"></a>
## 3. リモートトレーニングシステムの作成

今、あなたはリモートトレーニングシステム(RTS)を作成することを学びます。RTSは、トレーニングを実行するためのアグリゲータへの複数の当事者の呼び出しの受信を処理します。
- allowed_identities` は、Federated Learning実験への接続を許可するユーザです。 このチュートリアルでは、あなたのユーザIDのみが接続を許可されていますが、必要に応じてテンプレートを更新してユーザを追加することができます。
- remote_admin`の管理者です。管理者のテンプレートはユーザと同じです。このチュートリアルでは、テンプレートAdminを作成します。これもユーザIDと同じですが、一般的にアプリケーションでは管理者はユーザの一人である必要はありません。

In [ ]:
wml_remote_training_system_asset_one_def = Template("""
{
  "name": "Remote Party 1",
  "project_id": "$projectId",
  "description": "Sample Remote Training System",
  "tags": [ "Federated Learning" ],
  "organization": {
    "name": "IBM",
    "region": "US"
  },
  "allowed_identities": [
    {
      "id": "$userID",
      "type": "user"
    }
  ],
  "remote_admin": {
    "id": "$userID",
    "type": "user"
  }
}
""").substitute(userID = CLOUD_USERID,
                projectId = PROJECT_ID)


wml_remote_training_system_one_resp = requests.post(WML_SERVICES_URL + "/ml/v4/remote_training_systems", 
                                                    headers={"Content-Type": "application/json",
                                                             "Authorization": token}, 
                                                    params={"version": API_VERSION,
                                                            "project_id": PROJECT_ID}, 
                                                    data=wml_remote_training_system_asset_one_def, 
                                                    verify=False)

print(wml_remote_training_system_one_resp)
status_json = json.loads(wml_remote_training_system_one_resp.content.decode("utf-8"))
print("Create wml_remote_training_system_one asset response: %s"  % json.dumps(status_json, indent=4))
wml_remote_training_system_one_asset_uid = status_json["metadata"]["id"]
print("WML wml_remote_training_system_one asset uid: %s" % wml_remote_training_system_one_asset_uid)

<a id = "fl-job"></a>
## 4. FLトレーニングジョブを作成

このセクションでは、Federated Learningの実験を開始します。


In [ ]:
training_payload = Template(""" 
{
  "name": "FL Aggregator",
  "tags": [
    {
      "value": "tags_jobs_fl",
      "description": "Sample FL Aggregator"
    }
  ],
  "federated_learning": {
    "model": {
      "spec": {
        "id": "$modelID"
      },
      "type": "tensorflow"
    },
    "fusion_type": "iter_avg",
    "rounds": 5,
    "remote_training" : {
      "quorum": 1.0,
      "remote_training_systems": [ { "id" : "$rts_one", "required" : true  } ]
    },
    "hardware_spec": {
      "name": "XS"
    }
  },
  "training_data_references": [],
  "results_reference": {
    "type": "s3",
    "name": "outputData",
    "connection": {
      "endpoint_url": "$endpoint",
      "access_key_id": "$accessId",
      "secret_access_key": "$secretKey"
    },
    "location": {
      "bucket": "$bucket",
      "path": "."
    }
  },
  "project_id": "$projectId"  
}
""").substitute(modelID = base_model_id,
                projectId = PROJECT_ID,
                rts_one = wml_remote_training_system_one_asset_uid,
                endpoint = COS_ENDPOINT_URL,
                accessId = COS_ACCESS_KEY_ID,
                secretKey = COS_SECRET_ACCESS_KEY,
                bucket = COS_BUCKET)

print("Training payload: %s" % training_payload)

training_launch_resp = requests.post(WML_SERVICES_URL + "/ml/v4/trainings", 
                                     headers={"Content-Type": "application/json",
                                              "Authorization": token},
                                     data=training_payload,
                                     verify=False)

print(training_launch_resp)
print("Launch training response %s"  % training_launch_resp.content.decode("utf-8"))
training_id = json.loads(training_launch_resp.content.decode("utf-8"))["metadata"]["id"]
print("Training ID: %s" % training_id)

<a id = "status"></a>
### 4.1  トレーニングジョブのステータス取得

In [ ]:
status_full = requests.get(WML_SERVICES_URL + "/ml/v4/trainings/" + training_id,
                           headers={"Content-Type": "application/json",
                                    "Authorization": token},
                           params={"version": API_VERSION,
                                   "project_id": PROJECT_ID},
                           verify=False)

print(status_full)
status_json = json.loads(status_full.content.decode("utf-8"))
print("Full training job status: "+ json.dumps(status_json, indent=4))

### アグリゲータのステータスが受け入れ側になるのを待つ

<div class="alert alert-block alert-warning">先に進む前に、前のセルを実行します。  
出力は `accepting_parties` であるはずです。  
出力が `pending` の場合は、数分待ってから <i>previous</i> のコードセルを <code>requests.get</code> で再実行して、このコードセルを再実行して出力を確認してください。</div>

### 変数を取得し、パーティーのノートブックに貼り付け

以下のセルを実行し、出力をコピーします。

In [ ]:
print("WML_SERVICES_HOST = '%s'" % WML_SERVICES_HOST)
print("IAM_APIKEY = '%s'" % IAM_APIKEY)
print("RTS_ID = '%s'" % wml_remote_training_system_one_asset_uid)
print("TRAINING_ID = '%s'" % (training_id))

管理者として、あなたは今、Federated Learningの実験を開始しました。  
前のセルの出力をコピーします。  
Part 2 - WML Federated Learning with MNIST for Partyを開き、最初のコードセルに出力を貼り付けてください。

<a id = "summary"></a> 
## まとめ

おめでとう！あなたは以下のことを学びました。

1. Fedareted Learningの実験を開始する
2. テンプレートモデルを読み込む
3. RTSを作成し、実験ジョブを起動する
4. 学習用のデータセットをロードする
5. データハンドラの定義
6. パーティーを設定する
7. アグリゲータに接続する
8. Federated Learningモデルをトレーニングする

### もっと詳しく知りたい方はこちら

- Federated Learningの設定、用語、UIからのFederated Learningの実行についての詳細は、<a href = "https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fed-lea.html?audience=wdp" target="_blank" rel="noopener no referrer">Federated Learning documentation</a> for Cloudを参照してください。
- Kerasのモデルテンプレートの詳細については、<a href = "https://www.tensorflow.org/tutorials/quickstart/advanced" target="_blank" rel="noopener no referrer">こちら</a>を参照してください。
